# Training RNN on a book
Here Time Machine
The special thing  
Here is the input format, for each time-step generate a batch that contains the inputs


In [8]:
import torch as th
a = th.tensor([-40, 12, 12, 16, 8, 11.6])
a = th.tensor([0, 0, 0, 0])
b = th.softmax(a,0)
th.argmax(b, 0)

RuntimeError: "softmax_lastdim_kernel_impl" not implemented for 'Long'

In [49]:
import math
import torch as th
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [50]:
class RNNScratch(d2l.Module):
    def __init__(self, num_inputs: int, num_hiddens: int, sigma=0.01) -> None:
        super(RNNScratch, self).__init__()
        self.save_hyperparameters()
        self.W_xh = nn.Parameter(th.randn(num_inputs, num_hiddens) * sigma)
        self.W_hh = nn.Parameter(th.randn(num_hiddens, num_hiddens) * sigma)
        self.b_h = nn.Parameter(th.zeros(num_hiddens))

In [51]:
from typing import Optional
@d2l.add_to_class(RNNScratch)
def forward(self, inputs: th.tensor, state: Optional[th.tensor] = None):
    if state is None:
        # Initial state with shape: (batch_size, num_hiddens)
        state = th.zeros((inputs.shape[1], self.num_hiddens), device=inputs.device)
    else:
        state, = state  # todo is state an iterable? is it a tuple?
    outputs = []
    for X in inputs:    # Shape of inputs: (num_steps, batch_size, num_inputs)
        state = th.tanh(X @ self.W_xh + state @ self.W_hh + self.b_h)
        outputs.append(state)
    return outputs, state

In [52]:
num_steps, batch_size, num_inputs, num_hiddens = 100, 2, 16, 32
rnn = RNNScratch(num_inputs, num_hiddens)
# Since now the X is X_t
X = th.ones((num_steps, batch_size, num_inputs))
outputs, state = rnn(X)
outputs[0].shape, state.shape

(torch.Size([2, 32]), torch.Size([2, 32]))

In [53]:
def check_len(a, n):  #@save
    """Check the length of a list."""
    assert len(a) == n, f'list\'s length {len(a)} != expected length {n}'

def check_shape(a, shape):  #@save
    """Check the shape of a tensor."""
    assert a.shape == shape, \
            f'tensor\'s shape {a.shape} != expected shape {shape}'

check_len(outputs, num_steps)
check_shape(outputs[0], (batch_size, num_hiddens))
check_shape(state, (batch_size, num_hiddens))

In [54]:
class RNNLMScratch(d2l.Classifier):  #@save
    """The RNN-based language model implemented from scratch.
    The main extra is that it generated the output"""
    def __init__(self, rnn, vocab_size, lr=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.init_params()
    
    def init_params(self):  
        """Initializes the output weight parameters."""
        self.W_hq = nn.Parameter(
            th.randn(
                self.rnn.num_hiddens, self.vocab_size) * self.rnn.sigma)
        self.b_q = nn.Parameter(th.zeros(self.vocab_size))
    
    def training_step(self, batch):
        Y_hat = self(*batch[:-1])
        Y = batch[-1]
        l = self.loss(Y_hat, Y)
        self.plot('ppl', th.exp(l), train=True)
        return l
    
    def validation_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        self.plot('ppl', th.exp(l), train=False)


In [55]:
oh = F.one_hot(th.tensor([0, 2]), 5)
oh, oh.type(th.float32)

(tensor([[1, 0, 0, 0, 0],
         [0, 0, 1, 0, 0]]),
 tensor([[1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.]]))

In [56]:
@d2l.add_to_class(RNNLMScratch)  #@save
def one_hot(self, X):
    # Output shape: (num_steps, batch_size, vocab_size)
    # Make them float to be able to operate on them.
    return F.one_hot(X.T, self.vocab_size).type(th.float32)


In [57]:
oh = F.one_hot(th.tensor([0, 2]), 5)
oh.type(th.float32)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [58]:
@d2l.add_to_class(RNNLMScratch)
def output_layer(self, rnn_outputs: th.tensor):
    outputs = [H @ self.W_hq + self.b_q for H in rnn_outputs]
    return th.stack(outputs, dim=1)

@d2l.add_to_class(RNNLMScratch)
def forward(self, X: th.tensor, state: Optional[th.tensor] = None) -> th.tensor:
    embs = self.one_hot(X)
    rnn_outputs, _ = self.rnn(embs, state)
    return self.output_layer(rnn_outputs)

In [58]:
model = RNNLMScratch(rnn, num_inputs)
outputs = model(th.ones((batch_size, num_steps), dtype=th.int64))
check_shape(outputs, (batch_size, num_steps, num_inputs))
